In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.0 MB/s 


In [4]:
import pandas as pd
import sentencepiece as spm 
import urllib.request 
import csv
import numpy as np

In [13]:
data = pd.read_csv('구어체.csv', header=None)
data[:5]

,0,1
0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."


In [14]:
data[0] = data[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,0,1
0,설정을 입력하고 안정될 때까지 분 동안 기다린 후 버튼을 길게 누르십시오,"Enter into 0 setting, and wait for 5 minutes t..."
1,은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다,The zero was nothing for them but nothing coul...
2,버전에서 핫키 버그가 있습니다,"There is a Hotkey bug in the 1,015 version."
3,점에서 점을 득점한 사람은 재판을 위해 걸러집니다,"Individuals who got a score between 1,390 and ..."
4,년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을 것입니다,"Indeed, worshippers at the very first cathedra..."


In [15]:
data[0] = data[0].str.replace('^ +', "") # white space 데이터를 empty value로 변경
data[0].replace('', np.nan, inplace=True)
print(data.isnull().sum())

data = data.dropna(how = 'any')# Null 값이 존재하는 행 제거 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0    2
1    0
dtype: int64


In [16]:
# 결과를 data_kor.txt 파일에 저장
with open('data_k.txt', 'w', encoding='utf8') as f:
	f.write('\n'.join(data[0]))

In [17]:
data_k = pd.read_table('data_k.txt', header=None)
print(data_k)

                                                     0
0            설정을 입력하고 안정될 때까지 분 동안 기다린 후  버튼을 길게 누르십시오
1                     은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다
2                                     버전에서 핫키 버그가 있습니다
3                          점에서 점을 득점한 사람은 재판을 위해 걸러집니다
4       년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을 것입니다
...                                                ...
199993                           당신은 곧 당신의 조카를 볼 수 있어요
199994                          나는 당신이 내게 공유한 음악을 들었어요
199995                          그 서류는 당신이 공유해 준 것과 달라요
199996                             당신이 잡지에 광고 낸 것을 봤어요
199997                           당신이 이제 괜찮다니 저는 참 기쁘군요

[199998 rows x 1 columns]


In [20]:
spm.SentencePieceTrainer.Train('--input=data_k.txt --model_prefix=data_k --vocab_size=32000 --model_type=bpe --max_sentence_length=9999')

In [22]:
sp = spm.SentencePieceProcessor()
vocab_file = "data_k.model"
sp.load(vocab_file)

True

In [23]:
review = data_k[0]

lines = review[110500:110511].tolist()

for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

정말 신기하게도 공통점이 전혀 없는 형이에요
['▁정말', '▁신기', '하게도', '▁공통점이', '▁전혀', '▁없는', '▁형', '이에요']
[218, 5625, 5399, 19254, 1321, 598, 551, 96]

정말 심각하게 좋은 날이네요
['▁정말', '▁심각하게', '▁좋은', '▁날이', '네요']
[218, 11455, 222, 2484, 355]

정말 심각한 문제는 이들이 식물의 뿌리를 갉아 먹어서 식물의 뿌리에 알을 낳는 붕어가 알을 낳지 못해요
['▁정말', '▁심각한', '▁문제는', '▁이들이', '▁식물의', '▁뿌리를', '▁', '갉', '아', '▁먹어서', '▁식물의', '▁뿌', '리에', '▁알을', '▁낳', '는', '▁붕', '어가', '▁알을', '▁낳', '지', '▁못해요']
[218, 3610, 1907, 9368, 28679, 22679, 30781, 0, 30807, 10495, 28679, 2633, 962, 19779, 4593, 30784, 9446, 1243, 19779, 4593, 30791, 3555]

정말 아름다운 전시회입니다
['▁정말', '▁아름다운', '▁전시회', '입니다']
[218, 1542, 8461, 20]

정말 염치없지만 말하고 싶은 게 있습니다
['▁정말', '▁염', '치', '없', '지만', '▁말하고', '▁싶은', '▁게', '▁있습니다']
[218, 2471, 30916, 30873, 67, 3240, 616, 186, 41]

정말 예측 불가능하죠 그렇지 않아요
['▁정말', '▁예측', '▁불가능', '하죠', '▁그렇지', '▁않아요']
[218, 3295, 1817, 4287, 2625, 689]

정말 오랜 친구인 것처럼 느꼈고 자매처럼 보였어요
['▁정말', '▁오랜', '▁친구인', '▁것처럼', '▁느꼈고', '▁자매', '처럼', '▁보였어요']
[218, 1714, 14636, 1398, 19983, 19810, 440, 7541]
